# SMART examples
Let's experiment with some examples showing how to use a few of the endpoints of the SMART OAuth2 workflow. For these examples, the SMART server is accessible at <http://localhost:9000>, which is the default configuration when running the [S4S Docker reference stack](https://www.github.com/sync-for-science/reference-stack-docker).

## Client registration

A research application must register itself with the SMART server before it can access any data. This registration workflow is implemented as described in [RFC7591](https://tools.ietf.org/html/rfc7591). Client registration is performed by sending a POST request to the registration endpoint at <http://localhost:9000/oauth/register>. The body of the request is a JSON payload with the application's name, an array of redirect URIs, and the desired scopes. We'll keep the same redirect URI that is used by the reference stack's research application example.

In [1]:
import requests
data = {
    'client_name': 'SMART Test Client',
    'redirect_uris': ['http://localhost:9001/authorized'],
    'scope': 'launch/patient patient/*.read offline_access'
}
response = requests.post('http://localhost:9000/oauth/register', json=data)
response_data = response.json()
print(f'Response status code: {response.status_code}')

Response status code: 201


Success! Our client registration was completed successfully. The response is JSON containing the created registration paramters:

In [2]:
from pprint import pprint
pprint(response_data)

client_id = response_data['client_id']
client_secret = response_data['client_secret']

{'client_id': '59b2f830-965e-4a9d-be34-79283df73d24',
 'client_name': 'SMART Test Client',
 'client_secret': '337c78d3-0a12-42ad-b0fd-ed15d8e91800',
 'client_secret_expires_at': 0,
 'redirect_uris': ['http://localhost:9001/authorized'],
 'scope': 'launch/patient patient/*.read offline_access'}


We've been issued both a `client_id` and a `client_secret`, which will be used to identify the research application to the SMART server from now on.

## Token debug endpoints

Now that we have a client registered, let's use the token debug endpoints that are included with the S4S SMART server (for development purposes only).

### Create a token

We can create an access token for a registered client with a POST request to <http://localhost:9000/oauth/debug/token>. The request should include the client ID obtained from registration, the username of a user of the SMART server, and the patient ID which is associated with the user. The S4S SMART server ships with a user named `daniel-adams` which has access to a patient with ID `smart-1288992`, so we'll use these.

In [3]:
data = {
    'client_id': client_id,
    'username': 'daniel-adams',
    'patient_id': 'smart-1288992'
}
response = requests.post('http://localhost:9000/oauth/debug/token', json=data)
response_data = response.json()
print(f'Response status code: {response.status_code}')
pprint(response_data)

access_token = response_data['access_token']
refresh_token = response_data['refresh_token']

Response status code: 200
{'access_token': '5b7ba512-bbe7-47bd-999a-8458fdcb5bda',
 'refresh_token': 'cebae0fd-d604-4741-a581-5581b153d55f'}


### Inspect a token

Now that we have a token granting us access to a user, let's inspect it using a GET request to <http://localhost:9000/oauth/debug/introspect>:

In [4]:
params = {'token': access_token}  # can be access or refresh token
response = requests.get('http://localhost:9000/oauth/debug/introspect', params=params)
print(f'Response status code: {response.status_code}')
pprint(response.json())

Response status code: 200
{'access_expires': 'Thu, 05 Jul 2018 17:41:24 GMT',
 'access_token': '5b7ba512-bbe7-47bd-999a-8458fdcb5bda',
 'active': True,
 'approval_expires': 'Fri, 05 Jul 2019 16:41:24 GMT',
 'client_id': '59b2f830-965e-4a9d-be34-79283df73d24',
 'refresh_token': 'cebae0fd-d604-4741-a581-5581b153d55f',
 'scope': 'launch/patient patient/*.read offline_access',
 'security_labels': [],
 'token_type': 'Bearer',
 'username': 'daniel-adams'}


Here we can see information associated with this token. By default, the _access token_ is valid for 1 hour (`access_expires`), but the approval is valid for 1 year (`approval_expires`). This means that after 1 hour, attempts to use the `access_token` when fetching data will fail; however, the approval is still valid for 1 year, so the `refresh_token` may be used to generate a new access token within this time frame. For debugging purposes, these parameters can be specified in the request to the debug token endpoint:

In [5]:
from time import time
data = {
    'client_id': client_id,
    'username': 'daniel-adams',
    'patient_id': 'smart-1288992',
    'access_lifetime': 3*60*60,  # duration in seconds
    'approval_expires': time() + 180*24*60*60  # UNIX timestamp
}
token_response = requests.post('http://localhost:9000/oauth/debug/token', json=data)
access_token = token_response.json()['access_token']
introspect_response = requests.get('http://localhost:9000/oauth/debug/introspect',
                                   params={'token': access_token})
pprint(introspect_response.json())

{'access_expires': 'Thu, 05 Jul 2018 19:41:28 GMT',
 'access_token': 'a3954277-3505-4e24-8904-b6bdfe1ab149',
 'active': True,
 'approval_expires': 'Tue, 01 Jan 2019 16:41:26 GMT',
 'client_id': '59b2f830-965e-4a9d-be34-79283df73d24',
 'refresh_token': '2f9f60ee-2317-4517-90f0-7e049e4185c4',
 'scope': 'launch/patient patient/*.read offline_access',
 'security_labels': [],
 'token_type': 'Bearer',
 'username': 'daniel-adams'}


The `access_expires` and `approval_expires` reflect the requested durations of 3 hours and 180 days, respectively.

## Using the tokens to retreive data

Now that we have a client ID, client secret, and access token, we can use these values to get some FHIR data, which our application can store. The access token is simply used as a bearer token in the header of the request to the SMART server. In the S4S reference stack implementation, the SMART server then proxies the authenticated request to the HAPI-FHIR server. Normally, the request will fail if you try to access the patient data without the access token:

In [6]:
response = requests.get('http://localhost:9000/api/fhir/Patient/smart-1288992')  # oops, no header
print(f'Response status code: {response.status_code}')

Response status code: 401


Now let's try the same request with the access token:

In [7]:
headers = {
    'Authorization': f'Bearer {access_token}'
}
response = requests.get('http://localhost:9000/api/fhir/Patient/smart-1288992', headers=headers)
print(f'Response status code: {response.status_code}')
pprint(response.json())

Response status code: 200
{'active': True,
 'address': [{'city': 'Tulsa',
              'country': 'USA',
              'line': ['1 Hill AveApt 14'],
              'postalCode': '74117',
              'state': 'OK',
              'use': 'home'}],
 'birthDate': '1925-12-23',
 'gender': 'male',
 'id': 'smart-1288992',
 'identifier': [{'system': 'http://hospital.smarthealthit.org',
                 'type': {'coding': [{'code': 'MR',
                                      'display': 'Medical record number',
                                      'system': 'http://hl7.org/fhir/v2/0203'}],
                          'text': 'Medical record number'},
                 'use': 'usual',
                 'value': '1288992'}],
 'meta': {'lastUpdated': '2018-07-05T16:39:38.468+00:00',
          'security': [{'code': 'patient',
                        'system': 'http://smarthealthit.org/security/categories'},
                       {'code': 'Patient/smart-1288992',
                        'system': 'htt

If the access token has expired, the refresh token can be used to generate a new one with a POST request to the token endpoint at <http://localhost:9000/oauth/token> (using basic authentication):

In [14]:
auth = (client_id, client_secret)
data = {
    'grant_type': 'refresh_token',
    'refresh_token': refresh_token
}
response = requests.post('http://localhost:9000/oauth/token', auth=auth, data=data)
print(response.text)
pprint(response.json())

{"error": "invalid_grant"}
{'error': 'invalid_grant'}


# OAuth Workflow Walkthrough

To demonstrate how to use the OAuth workflow, let's walk through each step. We'll need the FHIR server with the SMART layer provided by the reference stack, and we'll interact with it as if we are a research application. The steps are:
1. Register our application as a client
2. Launch the workflow by asking the SMART server for authorization
3. Collect the authorization code from the SMART server
4. Exchange the code for an access token
5. Use the access token to access patient data

## Registering the client

This is the same process outlined above, with the exception of the provided redirect URI. Normally, this should be an endpoint exposed by the research client which accepts authorization codes from the SMART server; however, we are manually emulating a research application, so we don't have any such endpoint available. Instead, we can just give it a fake endpoint and copy the code from the browser into the notebook at the end of the participant's authorization process.

In [9]:
import requests
from pprint import pprint

redirect_uri = 'http://not-a-real-site'

data = {
    'client_name': 'Fake Research Application',
    'redirect_uris': [redirect_uri],
    'scope': 'launch/patient patient/*.read offline_access'
}
response = requests.post('http://localhost:9000/oauth/register', json=data)
response_data = response.json()
print(f'Response status code: {response.status_code}')
pprint(response.json())

client_id = response_data['client_id']
client_secret = response_data['client_secret']

Response status code: 201
{'client_id': '8d26ab7b-9b48-41d7-be2f-cc0347656f1e',
 'client_name': 'Fake Research Application',
 'client_secret': '7b1c8673-55c9-44d0-8e3e-88015285c9fa',
 'client_secret_expires_at': 0,
 'redirect_uris': ['http://not-a-real-site'],
 'scope': 'launch/patient patient/*.read offline_access'}


## Launch the OAuth workflow

The workflow is lauched by the participant by accessing the authorize endpoint exposed by the SMART server, which can be determined by looking at the server's conformance statement. For the reference stack's SMART server, this endpoint is <http://localhost:9000/oauth/authorize>.

In [10]:
from urllib.parse import urlencode

params = {
    'response_type': 'code',
    'client_id': client_id,
    'redirect_uri': redirect_uri,
    'scope': 'launch/patient patient/*.read offline_access',
    'state': 'my-obscure-state',
    'aud': 'http://localhost:9000/api/fhir'
}

print('Continue to authorization:')
print(f'http://localhost:9000/oauth/authorize?{urlencode(params)}')

Continue to authorization:
http://localhost:9000/oauth/authorize?response_type=code&client_id=8d26ab7b-9b48-41d7-be2f-cc0347656f1e&redirect_uri=http%3A%2F%2Fnot-a-real-site&scope=launch%2Fpatient+patient%2F%2A.read+offline_access&state=my-obscure-state&aud=http%3A%2F%2Flocalhost%3A9000%2Fapi%2Ffhir


## Collect the authorization code

By following that link, you may be asked to login with the default credentials (or you may already be logged in), and then asked to complete the authorization process. At the end of the process, you'll be redirected to a URL like <http://not-a-real-site/?code=LmM28GpIYoF4kULyRWcb7W4Fqlximv&state=my-obscure-state>, containing the authorization code. Copy this code into the script below, but **act fast** since this code expires quickly!

In [11]:
code = 'sm0E77sNswaVnsknQTBhiEbIRFrH7h'  # replace with your code

## Exchange for access token

With code in hand, we can now get an access token from the SMART server. This is completed by your application without any input from the user. Note that since this is a *confidential* client, we need to use basic authentication when interacting with the SMART server.

In [12]:
auth = (client_id, client_secret)  # for basic authentication

data = {
    'grant_type': 'authorization_code',
    'code': code,
    'redirect_uri': redirect_uri
}

response = requests.post('http://localhost:9000/oauth/token', auth=auth, data=data)
response_data = response.json()

print(f'Response status code: {response.status_code}')
pprint(response_data)

access_token = response_data['access_token']
patient_id = response_data['patient']

Response status code: 200
{'access_token': 'UxWwgbVdwc2TMyMXTRJAjGShoP98rT',
 'expires_in': 3600,
 'patient': 'smart-1288992',
 'refresh_token': 'xZkZAKpi79PMo4JefeThIi4At2SQ8A',
 'scope': 'launch/patient patient/*.read offline_access',
 'token_type': 'Bearer'}


**Note:** If you receive a 401 response, you may have taken too much time before the participant's authorization and this step.

You can see that the server gave us the patient ID whose data we now have access to, and the access token we can use to get the data.

## Accessing patient data

Now let's make sure we can access the data as we have above.

In [13]:
headers = {
    'Authorization': f'Bearer {access_token}'
}
response = requests.get(f'http://localhost:9000/api/fhir/Patient/{patient_id}', headers=headers)
print(f'Response status code: {response.status_code}')
pprint(response.json())

Response status code: 200
{'active': True,
 'address': [{'city': 'Tulsa',
              'country': 'USA',
              'line': ['1 Hill AveApt 14'],
              'postalCode': '74117',
              'state': 'OK',
              'use': 'home'}],
 'birthDate': '1925-12-23',
 'gender': 'male',
 'id': 'smart-1288992',
 'identifier': [{'system': 'http://hospital.smarthealthit.org',
                 'type': {'coding': [{'code': 'MR',
                                      'display': 'Medical record number',
                                      'system': 'http://hl7.org/fhir/v2/0203'}],
                          'text': 'Medical record number'},
                 'use': 'usual',
                 'value': '1288992'}],
 'meta': {'lastUpdated': '2018-07-05T16:39:38.468+00:00',
          'security': [{'code': 'patient',
                        'system': 'http://smarthealthit.org/security/categories'},
                       {'code': 'Patient/smart-1288992',
                        'system': 'htt

Success!